# Analyse des Températures Journalières et Anomalies Thermiques en France

L'objectif de cette analyse est de préparer les données météorologiques journalières pour étudier les températures maximales journalières en France, en calculant la moyenne glissante sur 30 jours et en déterminant les anomalies thermiques. Les données proviennent du fichier `Q_95_latest-2024-2025_RR-T-Vent.csv.gz` disponible sur le site [data.gouv.fr](https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_95_latest-2024-2025_RR-T-Vent.csv.gz), qui contient des informations météorologiques quotidiennes pour diverses stations en France.

Ce fichier de données est précieux pour comprendre les variations climatiques et météorologiques en France. L'analyse se concentre sur la température moyenne journalière (`TM`), le calcul de la moyenne glissante sur 30 jours, et l'identification des anomalies en comparant les valeurs journalières avec cette moyenne glissante. Ensuite, ces anomalies sont lissées sur 7 jours pour une meilleure visualisation.


## Méthodologie

La méthodologie utilisée implique plusieurs étapes clés :
1. Chargement et préparation des données : Le fichier CSV est chargé et les données sont nettoyées et préparées pour l'analyse.
2. Calcul de la moyenne glissante : Une requête SQL utilisant DuckDB est exécutée pour calculer la moyenne glissante sur 30 jours de la température moyenne journalière.
3. Détermination des anomalies : Les anomalies thermiques sont déterminées en comparant les températures journalières avec la moyenne glissante sur 30 jours.
4. Lissage des anomalies : Les anomalies sont ensuite lissées sur 7 jours pour réduire le bruit et améliorer la visualisation.
5. Visualisation : Les résultats sont visualisés à l'aide de Plotly, avec des graphiques montrant les températures journalières, la moyenne glissante, les anomalies, et les anomalies lissées.

Les données sont analysées à l'aide de requêtes SQL dans DuckDB, puis visualisées avec Plotly pour offrir une représentation claire et interactive des résultats.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_95_latest-2024-2025_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" WITH 
daily_avg AS (
  SELECT 
    CAST(SUBSTRING("AAAAMMJJ", 1, 4) AS INTEGER) AS year,
    CAST(SUBSTRING("AAAAMMJJ", 5, 2) AS INTEGER) AS month,
    CAST(SUBSTRING("AAAAMMJJ", 7, 2) AS INTEGER) AS day,
    CAST("TM" AS DOUBLE) AS "TM",
    "AAAAMMJJ"
  FROM 
    loaded_dataset
),
avg_tm AS (
  SELECT 
    "AAAAMMJJ",
    AVG("TM") AS avg_tm_day
  FROM 
    daily_avg
  GROUP BY 
    "AAAAMMJJ"
),
moving_avg AS (
  SELECT 
    "AAAAMMJJ",
    avg_tm_day,
    AVG(avg_tm_day) OVER (ORDER BY "AAAAMMJJ" ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS moving_avg_30_days
  FROM 
    avg_tm
),
anomalies AS (
  SELECT 
    "AAAAMMJJ",
    avg_tm_day,
    moving_avg_30_days,
    avg_tm_day - moving_avg_30_days AS anomaly
  FROM 
    moving_avg
)
SELECT 
  "AAAAMMJJ",
  avg_tm_day,
  moving_avg_30_days,
  anomaly,
  AVG(anomaly) OVER (ORDER BY "AAAAMMJJ" ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING) AS smoothed_anomaly_7_days
FROM 
  anomalies
ORDER BY 
  "AAAAMMJJ" """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui permet de créer des visualisations interactives et personnalisées. Cette technologie est adaptée pour représenter les températures journalières et les anomalies thermiques car elle permet de superposer plusieurs tracés et d'afficher des informations détaillées au survol. Cela facilite l'analyse et la compréhension des données.

In [3]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df['AAAAMMJJ'] = pd.to_datetime(df['AAAAMMJJ'], format='%Y%m%d')

dataviz = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    row_heights=[0.7, 0.3]
)

temp = go.Scatter(
    x=df['AAAAMMJJ'], y=df['avg_tm_day'], mode='lines',
    name='Température journalière', line=dict(color='#1f77b4', width=0.8),
    hovertemplate='T° : %{y:.2f}°C<br>%{x|%Y-%m-%d}<extra></extra>'
)
dataviz.add_trace(temp, row=1, col=1)

moy = go.Scatter(
    x=df['AAAAMMJJ'], y=df['moving_avg_30_days'], mode='lines',
    name='Moyenne glissante 30 jours', line=dict(color='#ff7f0e', width=2),
    hovertemplate='Moyenne : %{y:.2f}°C<br>%{x|%Y-%m-%d}<extra></extra>'
)
dataviz.add_trace(moy, row=1, col=1)

anom = go.Scatter(
    x=df['AAAAMMJJ'], y=df['anomaly'], mode='markers',
    name='Anomalie', marker=dict(color='rgba(31,119,180,0.4)', size=4),
    hovertemplate='Anom. : %{y:.2f}°C<br>%{x|%Y-%m-%d}<extra></extra>'
)
dataviz.add_trace(anom, row=2, col=1)

anom_lisse = go.Scatter(
    x=df['AAAAMMJJ'], y=df['smoothed_anomaly_7_days'], mode='lines',
    name='Anomalie lissée', line=dict(color='crimson', width=2),
    hovertemplate='Anom. lis. : %{y:.2f}°C<br>%{x|%Y-%m-%d}<extra></extra>'
)
dataviz.add_trace(anom_lisse, row=2, col=1)

mask = abs(df['anomaly']) > 5
points = df[mask]
if not points.empty:
    dataviz.add_trace(go.Scatter(
        x=points['AAAAMMJJ'], y=points['avg_tm_day'], mode='markers',
        marker=dict(color='#d62728', size=10, symbol='star'),
        name='Anomalie > 5°C',
        hovertemplate='T° : %{y:.2f}°C<br>Anomalie : %{customdata:.2f}°C<br>%{x|%Y-%m-%d}<extra></extra>',
        customdata=points['anomaly']
    ), row=1, col=1)

dataviz.update_layout(
    height=700,
    margin=dict(l=40, r=20, t=60, b=40),
    hovermode='x unified',
    font=dict(family='Source Sans Pro', size=12),
    title=dict(text="Températures journalières &anomalies thermiques", x=0.05),
    legend=dict(orientation="h", y=-0.05, x=0, xanchor='left'),
    xaxis2=dict(title="Date", title_standoff=10),
    yaxis=dict(title="Température (°C)"),
    yaxis2=dict(title="Anomalie (°C)")
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*